# Homework 1. Relational Algebra

1. Use Google Colabortory and do your homework.
3. (In Google colaboratory) Before you submit your homework, restart kernel and run all. (Runtime-> restart and run all)
4. Save (File->Save) the file
5. Submit your homework (this file) in Google classroom
6. **Don't forget to click "제출" button** ("Submit", "완료로 표시", 또는 "제출" 버턴을 누르지 않으면 제출된 것이 아님)
7. No late homeworks will be accepted for any reason!

## CompanyDB Schema

### Note that attribute names are capitalized
```
E(Fname,Minit,Lname,Ssn,Bdate,Address,Sex,Salary,Superssn,Dno)
D(Dname,Dnumber,Mgrssn,Mgrstartdate)
DL(Dnumber,Dlocation)
P(Pname,Pnumber,Plocation,Dnum)
WO(Essn,Pno,Hours)
DE(Essn,Dependent_name,Sex,Bdate,Relationship)
```

## First, run the following cell before you do your homework

## DB setup

In [1]:
EMPLOYEE_csv = 'Fname,Minit,Lname,Ssn,Bdate,Address,Sex,Salary,Superssn,Dno\r\nJohn,B,Smith,123456789,1965-01-09,731-Fondren-Houston-TX,M,30000.00,333445555,5\r\nFranklin,T,Wong,333445555,1955-12-08,638-Voss-Houston-TX,M,40000.00,888665555,5\r\nJoyce,A,English,453453453,1972-07-31,5631-Rice-Houston-TX,F,25000.00,333445555,5\r\nRamesh,K,Narayan,666884444,1962-09-15,975-Fire-Oak-Humble-TX,M,38000.00,333445555,5\r\nJames,E,Borg,888665555,1937-11-10,450-Stone-Houston-TX,M,55000.00,,1\r\nJennifer,S,Wallace,987654321,1941-06-20,291-Berry-Bellaire-TX,F,43000.00,888665555,4\r\nAhmad,V,Jabbar,987987987,1969-03-29,980-Dallas-Houston-TX,M,25000.00,987654321,4\r\nAlicia,J,Zelaya,999887777,1968-01-19,3321-Castle-Spring-TX,F,25000.00,987654321,4\r\n'
DEPARTMENT_csv = 'Dname,Dnumber,Mgrssn,Mgrstartdate\r\nHeadquarters,1,888665555,1981-06-19\r\nAdministration,4,987654321,1995-01-01\r\nResearch,5,333445555,1988-05-22\r\n'
DEPT_LOCATIONS_csv = 'Dnumber,Dlocation\r\n1,Houston\r\n4,Stafford\r\n5,Bellaire\r\n5,Houston\r\n5,Sugarland\r\n'
PROJECT_csv = 'Pname,Pnumber,Plocation,Dnum\r\nProductX,1,Bellaire,5\r\nProductY,2,Sugarland,5\r\nProductZ,3,Houston,5\r\nComputerization,10,Stafford,4\r\nReorganization,20,Houston,1\r\nNewbenefits,30,Stafford,4\r\n'
WORKS_ON_csv = 'Essn,Pno,Hours\r\n123456789,1,32.5\r\n123456789,2,7.5\r\n333445555,2,10.0\r\n333445555,3,10.0\r\n333445555,10,10.0\r\n333445555,20,10.0\r\n453453453,1,20.0\r\n453453453,2,20.0\r\n666884444,3,40.0\r\n888665555,20,\r\n987654321,20,15.0\r\n987654321,30,20.0\r\n987987987,10,35.0\r\n987987987,30,5.0\r\n999887777,10,10.0\r\n999887777,30,30.0\r\n'
DEPENDENT_csv = 'Essn,Dependent_name,Sex,Bdate,Relationship\r\n123456789,Alice,F,1988-12-30,Daughter\r\n123456789,Elizabeth,F,1967-05-05,Spouse\r\n123456789,Michael,M,1988-01-04,Son\r\n333445555,Alice,F,1986-04-05,Daughter\r\n333445555,Joy,F,1958-05-03,Spouse\r\n333445555,Theodore,M,1983-10-25,Son\r\n987654321,Abner,M,1942-02-28,Spouse\r\n'

# import packages

import pandas as pd
from io import StringIO 
import re

# Tables
EMPLOYEE = pd.read_csv(StringIO(EMPLOYEE_csv))
DEPARTMENT = pd.read_csv(StringIO(DEPARTMENT_csv))
DEPT_LOCATIONS = pd.read_csv(StringIO(DEPT_LOCATIONS_csv))
PROJECT = pd.read_csv(StringIO(PROJECT_csv))
WORKS_ON = pd.read_csv(StringIO(WORKS_ON_csv))
DEPENDENT = pd.read_csv(StringIO(DEPENDENT_csv))

# short_names for Tables
E = EMPLOYEE
D = DEPARTMENT
DL = DEPT_LOCATIONS
P = PROJECT
WO = WORKS_ON
DE = DEPENDENT

def rename(R, C):
    return R.rename(columns=C)

def select(R, C):
    # The following substitution may NOT work 
    # when column name is equal to a string value in C
    tokens = re.split(r'(\W+)', C)
    new_tokens = ["R." + token  if token in R.columns else token \
                  for token in tokens] 
    selected = eval(''.join(new_tokens))
    return R[selected].reset_index(drop=True)

def project(R, C):
    return R[C].drop_duplicates().reset_index(drop=True)

def union(R, S):
    S.columns = R.columns
    T = pd.concat([R, S], ignore_index=True)
    T = T.drop_duplicates().reset_index(drop=True)
    return T

def intersect(R, S):
    S.columns = R.columns
    return R.merge(S).reset_index(drop=True)

def minus(R, S):
    S.columns = R.columns
    return pd.concat([R, S, S]).drop_duplicates(keep=False).reset_index(drop=True)

def product(R, S):
    """Determine Cartesian product of two data frames."""
    key = 'key'
    while key in R.columns or key in S.columns:
        key = '_' + key
    key_d = {key: 0}
    return pd.merge(R.assign(**key_d), S.assign(**key_d), on=key)\
           .drop(key, axis=1).reset_index(drop=True)

def natural_join(R, S):
    return pd.merge(R, S).reset_index(drop=True)

def natural_join2(R, S, keys_R, keys_S):
    return pd.merge(R, S, left_on=keys_R, right_on=keys_S)\
           .drop(keys_S, axis=1).reset_index(drop=True)

def division(R, S):
    Z = set(R.columns)
    X = set(S.columns)
    assert X <= Z
    Y = pd.Index(Z - X)
    T1 = project(R, Y)
    T2 = project(minus(product(T1, S), R), Y)
    T = minus(T1, T2)
    return T.reset_index(drop=True)

## <font color='red'>Query Statement에 나타나지 않은 상수(값)을 사용하면 무조건 0점 처리됨. 예를 들어, Query 1에서 "Research"라는 상수값만 나타나야 함</font>

## 각 질의당 10점

### Query 1: Retrieve the last name, first name, and address of employees who work for the "Research" department. (Use cartesian product. Do NOT use join)

In [14]:
# YOUR CODE HERE      0
At = ["Lname", "Fname", "Address"]
pt = product(E, D) # 14 column 10+4
T = project(select(product(E, D), "Dname == 'Research'"),At)
T

,Lname,Fname,Address
0,Smith,John,731-Fondren-Houston-TX
1,Wong,Franklin,638-Voss-Houston-TX
2,English,Joyce,5631-Rice-Houston-TX
3,Narayan,Ramesh,975-Fire-Oak-Humble-TX
4,Borg,James,450-Stone-Houston-TX
5,Wallace,Jennifer,291-Berry-Bellaire-TX
6,Jabbar,Ahmad,980-Dallas-Houston-TX
7,Zelaya,Alicia,3321-Castle-Spring-TX


### Query 2: Retrieve the last name, first name, and address of employees who work for the "Research" department. (Do NOT use cartesian product. Use natural join)

In [3]:
# YOUR CODE HERE      10
At = ["Lname", "Fname", "Address"]
D = rename(D, {'Dnumber':'Dno'})
T = natural_join(E, D)
S = select(T, "Dname == 'Research'")
R = project(S, At)
R

,Lname,Fname,Address
0,Smith,John,731-Fondren-Houston-TX
1,Wong,Franklin,638-Voss-Houston-TX
2,English,Joyce,5631-Rice-Houston-TX
3,Narayan,Ramesh,975-Fire-Oak-Humble-TX


### Query 3: For every project located in "Stafford", list the project name, the controlling department name, and the department manager's last name, first name. (Use natural_join2, No cartesian products and natural joins are allowed)

In [4]:
# YOUR CODE HERE      10
L = natural_join2(P, D, ['Dnum'], ['Dno'])
S = select(L, "Plocation == 'Stafford'")
K = natural_join2(E, S, ['Ssn'], ['Mgrssn'])
R = project(K, ['Pname', 'Dname', 'Lname', 'Fname'])
R

,Pname,Dname,Lname,Fname
0,Computerization,Administration,Wallace,Jennifer
1,Newbenefits,Administration,Wallace,Jennifer


### Query 4: Find the last name, first name of employees who work on all the projects controlled by department number 4.

In [5]:
# YOUR CODE HERE      0
At = ["Lname", "Fname"]
T = product(product(WO, P), E)
S = select(select(select(T, "Pno==Pnumber"), "Dnum==4"), "Essn==Ssn")
R = project(S, At)
R

,Lname,Fname
0,Wong,Franklin
1,Wallace,Jennifer
2,Jabbar,Ahmad
3,Zelaya,Alicia


### Query 5: Make a list of project names for projects that involve an employee whose last name is "Wallace", either as a worker or as a manager of the department that controls the project.

In [6]:
# YOUR CODE HERE      10
Em = natural_join2(project(E, 'Superssn'), E, ['Superssn'], ['Ssn'])
Dm = natural_join2(project(D, 'Mgrssn'), E, ['Mgrssn'], ['Ssn'])
Sp = union(Em, Dm)
Pn = project(select(Sp, "Lname == 'Wallace'"), 'Superssn_x')
Dn = project(select(Sp, "Lname == 'Wallace'"), 'Dno')
Ep = project(natural_join2(Pn, WO, ['Superssn_x'], ['Essn']), 'Pno')
Dp = project(natural_join2(Dn, P, ['Dno'], ['Dnum']), 'Pnumber')
R1 = project(natural_join2(Ep, P, ['Pno'], ['Pnumber']), ['Pname', 'Pno'])
R2 = project(natural_join(Dp, P), ['Pname', 'Pnumber'])
R = project(union(R1, R2), 'Pname')
R

0     Reorganization
1        Newbenefits
2    Computerization
Name: Pname, dtype: object

### Query 6: List the last name, first name of all employees with two or more dependents. (Do NOT use aggregate operator)

In [7]:
# YOUR CODE HERE      10
DT = product(DE, DE)
R = select(DT, 'Essn_x == Essn_y')
RR = project(select(R, 'Dependent_name_x != Dependent_name_y'), ['Essn_x'])
Re = project(natural_join2(RR, E, ['Essn_x'], ['Ssn']), ['Lname', 'Fname'])
Re

,Lname,Fname
0,Smith,John
1,Wong,Franklin


### Query 7: Retrieve the last name, first name of employees who have no dependents.

In [8]:
# YOUR CODE HERE        10
HD = project(natural_join2(project(DE, 'Essn'), E, ['Essn'], ['Ssn']), ['Lname', 'Fname', 'Essn'])
AE = project(E, ['Lname', 'Fname', 'Ssn'])
R = project(minus(AE, HD), ['Lname', 'Fname'])
R

,Lname,Fname
0,English,Joyce
1,Narayan,Ramesh
2,Borg,James
3,Jabbar,Ahmad
4,Zelaya,Alicia


### Query 8: List the last name, first name of managers who have at least one dependent.

In [9]:
# YOUR CODE HERE      5 projection
HD = project(natural_join2(E, project(DE, ['Essn']), ['Ssn'], ['Essn']), ['Lname', 'Fname', 'Ssn'])
Ms = project(E, ['Superssn'])
Ms = rename(Ms,{'Superssn':'Ssn'})
AM = project(natural_join(E, Ms), ['Lname', 'Fname', 'Ssn'])
intersect(HD, AM)

,Lname,Fname,Ssn
0,Wong,Franklin,333445555
1,Wallace,Jennifer,987654321


### Query 9: Retrieve the last name, first name of all employees in department 5 who work more than 10 hours per week on the ProductX project.

In [10]:
# YOUR CODE HERE      10
AE = project(select(E, "Dno==5"), ['Lname', 'Fname', 'Ssn'])
PI = natural_join2(WO, P, ['Pno'], ['Pnumber'])
IN = natural_join2(AE, PI, ['Ssn'], ['Essn'])
project(select(IN, "(Pname=='ProductX') & (Hours>10)"), ['Lname', 'Fname'])

,Lname,Fname
0,Smith,John
1,English,Joyce


### Query 10: Retrieve the last name, first name of all employees who is supervised by James Borg

In [11]:
# YOUR CODE HERE      10
JB = project(select(E, "(Fname=='James') & (Lname=='Borg')"), ['Ssn'])
AE = project(natural_join2(JB, E, ['Ssn'], ['Superssn']), ['Lname', 'Fname'])
AE

,Lname,Fname
0,Wong,Franklin
1,Wallace,Jennifer
